In [48]:
import os
import json
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from typing import List, Dict, Any, Tuple, Optional, Union
from pinecone import Pinecone, ServerlessSpec
import logging
from dataclasses import dataclass, asdict
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import torch
import numpy as np
import cv2
from PIL import Image


from typing import List, Optional
from pydantic import BaseModel, Field
from llama_index.core import (
    VectorStoreIndex,
    Document,
    StorageContext,
    Settings,
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.schema import NodeWithScore


import io
import pymupdf as fitz
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from llama_index.core import Document, Settings, StorageContext, VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition
import logging
from llama_index.core import Settings, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.readers.file import PDFReader
from datetime import datetime
from llama_index.core import Document, Settings, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings, VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI
from llama_parse import LlamaParse
from typing import Sequence
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition
from dataclasses import dataclass
from typing import List, Dict, Any
import pymupdf as fitz
import llama_index.core
os.environ["OPENAI_API_KEY"] = "xxxxxxx"

In [49]:
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)
embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-base-en-v1.5",
            embed_batch_size=10
)
Settings.embed_model = embed_model
Settings.dimension = 768
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('document_processing.log'),
        logging.StreamHandler()
    ]
)

2024-11-04 10:15:38,159 - opentelemetry.instrumentation.instrumentor - WARNING - Attempting to instrument while already instrumented
2024-11-04 10:15:38,166 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
2024-11-04 10:15:44,169 - sentence_transformers.SentenceTransformer - INFO - 2 prompts are loaded, with the keys: ['query', 'text']


In [53]:
from typing import List, Optional
from pydantic import BaseModel, Field
from llama_index.core import (
    VectorStoreIndex,
    Document,
    StorageContext,
    Settings,
    load_index_from_storage,
)
from llama_index.core.vector_stores.simple import SimpleVectorStore

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.schema import NodeWithScore
mcd = None
class TextBlock(BaseModel):
    """Text block in the report"""
    text: str = Field(..., description="The text content")
    source: Optional[str] = Field(None, description="Source of the text")
    page_num: Optional[int] = Field(None, description="Page number")

class ImageBlock(BaseModel):
    """Image block in the report"""
    file_path: str = Field(..., description="Path to the image file")
    caption: Optional[str] = Field(None, description="Image caption")
    source: Optional[str] = Field(None, description="Source of the image")
    page_num: Optional[int] = Field(None, description="Page number")

class TableBlock(BaseModel):
    """Table block in the report"""
    file_path: str = Field(..., description="Path to the table image")
    excel_path: Optional[str] = Field(None, description="Path to Excel file")
    caption: Optional[str] = Field(None, description="Table caption")
    source: Optional[str] = Field(None, description="Source of the table")
    page_num: Optional[int] = Field(None, description="Page number")

class ReportOutput(BaseModel):
    """Structured output for report generation"""
    title: str = Field(..., description="Report title")
    summary: str = Field(..., description="Executive summary")
    blocks: List[Union[TextBlock, ImageBlock, TableBlock]] = Field(
        ..., 
        description="Content blocks in the report"
    )

class RAGQueryEngine:
    """Enhanced RAG Query Engine with structured outputs"""
    
    def __init__(self, storage_dir: str = "./storage"):
        self.storage_dir = Path(storage_dir)
        self.logger = logging.getLogger(__name__)
        
        # Initialize embedding model
        self.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-base-en-v1.5",
            embed_batch_size=10
        )
        
        # Initialize LLM
        system_prompt = """You are a report generation assistant that produces well-formatted
        responses with text, images, and tables. Include relevant visual elements when they add value.
        Structure your response according to the ReportOutput schema."""
        
        self.llm = OpenAI(model="gpt-4o-mini", api_base="https://models.inference.ai.azure.com", system_prompt=system_prompt)
        
        # Set global models
        Settings.embed_model = self.embed_model
        Settings.llm = self.llm
        
        # Initialize storage context
        vector_store = SimpleVectorStore()
        self.storage_context = StorageContext.from_defaults(
            persist_dir=str(self.storage_dir),
            vector_store=vector_store
        )


    def build_index(self, documents: List[Document]):
        """Build or load index"""
        try:
            # Try to load existing index
            if (self.storage_dir / "index").exists():
                self.index = load_index_from_storage(self.storage_context)
                self.logger.info("Loaded existing index")
            else:
                # Build new index
                self.index = VectorStoreIndex.from_documents(
                    documents,
                    storage_context=self.storage_context,
                    show_progress=True
                )
                # Persist index
                self.index.storage_context.persist()
                self.logger.info("Built and persisted new index")
                
        except Exception as e:
            self.logger.error(f"Error building index: {str(e)}")
            raise

    async def query(
        self,
        query_text: str,
        response_mode: str = "tree_summarize",
        similarity_top_k: int = 5
            ) -> ReportOutput:
        """Execute structured query"""
        try:
            if not self.index:
                raise ValueError("Index not built. Please build index first.")
            
            # Create structured LLM
            structured_llm = self.llm.as_structured_llm(output_cls=ReportOutput)
            
            # Configure response synthesizer
            response_synthesizer = get_response_synthesizer(
                response_mode=response_mode,
                structured_answer_filtering=True
            )
            
            # Create query engine with all parameters
            query_engine = self.index.as_query_engine(
                similarity_top_k=similarity_top_k,
                response_synthesizer=response_synthesizer,
                llm=structured_llm,
                node_postprocessors=[
                    SimilarityPostprocessor(similarity_cutoff=0.7)
                ]
            )
            
            # Execute query
            response = await query_engine.aquery(query_text)
            
            # Process and structure the response
            return self._process_response(response)
            
        except Exception as e:
            self.logger.error(f"Error executing query: {str(e)}")
            raise

    def _process_response(self, response) -> ReportOutput:
        """Process query response into structured output"""
        blocks = []
        
        # Process source nodes
        for node in response.source_nodes:
            metadata = node.metadata
            
            if metadata.get("type") == "text":
                blocks.append(TextBlock(
                    text=node.text,
                    source=metadata.get("source"),
                    page_num=metadata.get("page_num")
                ))
            
            elif metadata.get("type") == "image":
                blocks.append(ImageBlock(
                    file_path=metadata.get("image"),
                    caption=metadata.get("caption"),
                    source=metadata.get("source"),
                    page_num=metadata.get("page_num")
                ))
            
            elif metadata.get("type") == "table":
                blocks.append(TableBlock(
                    file_path=metadata.get("image"),
                    excel_path=metadata.get("dataframe"),
                    caption=metadata.get("caption"),
                    source=metadata.get("source"),
                    page_num=metadata.get("page_num")
                ))
        mcd = response        
                
        logging.info(f"Processed response for query: {response.query}")
        logging.info(f"Response: {response}")
        
        return ReportOutput(
            title=f"Report",
            summary=response.response,
            blocks=blocks
        )

    def render_report(self, report: ReportOutput):
        """Render the structured report"""
        from IPython.display import display, Markdown, Image
        
        # Display title and summary
        display(Markdown(f"# {report.title}"))
        display(Markdown(f"## Executive Summary\n{report.summary}\n"))
        
        # Display content blocks
        for block in report.blocks:
            if isinstance(block, TextBlock):
                display(Markdown(f"### Content from {block.source} (Page {block.page_num})\n{block.text}\n"))
                
            elif isinstance(block, ImageBlock):
                display(Markdown(f"### Image from {block.source} (Page {block.page_num})"))
                display(Image(filename=block.file_path))
                if block.caption:
                    display(Markdown(f"*{block.caption}*\n"))
                    
            elif isinstance(block, TableBlock):
                display(Markdown(f"### Table from {block.source} (Page {block.page_num})"))
                display(Image(filename=block.file_path))
                if block.caption:
                    display(Markdown(f"*{block.caption}*\n"))
                display(Markdown(f"[Download Excel]({block.excel_path})\n"))


In [51]:
import logging
from pathlib import Path
from typing import List, Dict, Optional, Union
from dataclasses import dataclass
from pydantic import BaseModel, Field
import fitz
import torch
from transformers import AutoTokenizer, AutoModel
from IPython.display import display, Markdown, Image

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class ContentBlock(BaseModel):
    """Base content block"""
    content_type: str
    page_num: int
    metadata: Dict

class TextBlock(ContentBlock):
    """Text block for output"""
    text: str = Field(..., description="The text content")

class ImageBlock(ContentBlock):
    """Image block for output"""
    image_path: str = Field(..., description="Path to image file")
    caption: Optional[str] = Field(None, description="Image caption")

class ReportOutput(BaseModel):
    """Structured report output"""
    title: str = Field(..., description="Report title")
    summary: str = Field(..., description="Executive summary")
    blocks: List[Union[TextBlock, ImageBlock]] = Field(
        ..., description="Content blocks"
    )
    
    def render(self):
        """Render report with text and images"""
        display(Markdown(f"# {self.title}"))
        display(Markdown(f"## Executive Summary\n{self.summary}"))
        
        for block in self.blocks:
            if isinstance(block, TextBlock):
                display(Markdown(block.text))
            else:
                display(Image(filename=block.image_path))
                if block.caption:
                    display(Markdown(f"*{block.caption}*"))

class DocumentProcessor:
    """Document processing with PyMuPDF"""
    
    def __init__(self, storage_dir: str = "./storage"):
        self.storage_dir = Path(storage_dir)
        self.storage_dir.mkdir(exist_ok=True)
        
        # Create subdirectories
        self.image_dir = self.storage_dir / "images"
        self.image_dir.mkdir(exist_ok=True)
        
        self.table_dir = self.storage_dir / "tables"
        self.table_dir.mkdir(exist_ok=True)
        
        self.cache_dir = self.storage_dir / "cache"
        self.cache_dir.mkdir(exist_ok=True)
        
        self.logger = logging.getLogger(__name__)
        
    def process_document(self, file_path: str) -> List[Document]:
        """Process document and return list of Documents"""
        cache_path = self.cache_dir / f"{Path(file_path).stem}_processed.pkl"
        
        if cache_path.exists():
            self.logger.info(f"Loading cached processing results: {cache_path}")
            import pickle
            with open(cache_path, 'rb') as f:
                return pickle.load(f)
        
        try:
            doc = fitz.open(file_path)
            all_documents = []
            
            for page_num in range(len(doc)):
                page = doc[page_num]
                page_documents = self._process_page(page, page_num, file_path)
                all_documents.extend(page_documents)
            
            doc.close()
            
            # Cache results
            with open(cache_path, 'wb') as f:
                import pickle
                pickle.dump(all_documents, f)
            
            return all_documents
            
        except Exception as e:
            self.logger.error(f"Error processing document: {str(e)}")
            raise

    def _process_page(self, page, page_num: int, source_file: str) -> List[Document]:
        """Process a single page and return list of Documents"""
        documents = []
        
        try:
            # Extract text blocks
            text_blocks = self._extract_text_blocks(page, page_num)
            for idx, block in enumerate(text_blocks):
                doc = Document(
                    text=block["text"],
                    metadata={
                        "source": f"{Path(source_file).stem}-page{page_num}-text{idx}",
                        "page_num": page_num,
                        "type": "text",
                        "bbox": block["bbox"],
                        "id_": f"{Path(source_file).stem}-page{page_num}-text{idx}"
                    }
                )
                documents.append(doc)
            
            # Extract images
            images = self._extract_images(page, page_num, source_file)
            for idx, image in enumerate(images):
                doc = Document(
                    text=f"Image from page {page_num}",
                    metadata={
                        "source": f"{Path(source_file).stem}-page{page_num}-image{idx}",
                        "page_num": page_num,
                        "type": "image",
                        "image_path": image["image_path"],
                        "bbox": image["bbox"],
                        "id_": f"{Path(source_file).stem}-page{page_num}-image{idx}"
                    }
                )
                documents.append(doc)
            
            # Extract tables
            tables = self._extract_tables(page, page_num, source_file)
            for idx, table in enumerate(tables):
                doc = Document(
                    text=f"Table from page {page_num}",
                    metadata={
                        "source": f"{Path(source_file).stem}-page{page_num}-table{idx}",
                        "page_num": page_num,
                        "type": "table",
                        "table_path": table["table_path"],
                        "excel_path": table["excel_path"],
                        "bbox": table["bbox"],   
                        "id_": f"{Path(source_file).stem}-page{page_num}-table{idx}"                     
                    }
                )
                documents.append(doc)
            
            return documents
            
        except Exception as e:
            self.logger.error(f"Error processing page {page_num}: {str(e)}")
            raise
            
    def _extract_text_blocks(self, page, page_num: int) -> List[Dict]:
        """Extract text blocks from page"""
        blocks = []
        for block in page.get_text("blocks"):
            if block[6] == 0:  # Text block
                blocks.append({
                    "text": block[4],
                    "bbox": {
                        "x0": block[0],
                        "y0": block[1],
                        "x1": block[2],
                        "y1": block[3]
                    },
                    "page_num": page_num
                })
        return blocks
    
    def _extract_images(self, page, page_num: int, source_file: str) -> List[Dict]:
        """Extract images from page"""
        images = []
        for img_index, img in enumerate(page.get_images(full=True)):
            try:
                xref = img[0]
                base_image = page.parent.extract_image(xref)
                
                if base_image:
                    image_path = self.image_dir / f"{Path(source_file).stem}_page{page_num}_img{img_index}.png"
                    with open(image_path, "wb") as f:
                        f.write(base_image["image"])
                    
                    images.append(Document({
                        "image_path": str(image_path),
                        "bbox": img[1],
                        "page_num": page_num,
                        "id": img_index
                    }))
            except Exception as e:
                self.logger.warning(f"Error extracting image: {str(e)}")
                continue
                
        return images
    
    def _extract_tables(self, page, page_num: int, source_file: str) -> List[Dict]:
        """Extract tables from page"""
        tables = []
        try:
            found_tables = page.find_tables(
                horizontal_strategy="lines_strict",
                vertical_strategy="lines_strict"
            )
            
            for table_idx, table in enumerate(found_tables):
                try:
                    if not table.header.external:
                        # Convert to pandas DataFrame
                        pandas_df = table.to_pandas()
                        
                        # Save table as Excel
                        excel_path = self.table_dir / f"{Path(source_file).stem}_page{page_num}_table{table_idx}.xlsx"
                        pandas_df.to_excel(str(excel_path))
                        
                        # Create image of table area
                        table_bbox = fitz.Rect(table.bbox)
                        table_pix = page.get_pixmap(clip=table_bbox)
                        image_path = self.table_dir / f"{Path(source_file).stem}_page{page_num}_table{table_idx}.png"
                        table_pix.save(str(image_path))
                        
                        tables.append(Document({
                            "table_path": str(image_path),
                            "excel_path": str(excel_path),
                            "bbox": table.bbox,
                            "columns": list(pandas_df.columns),
                            "id_": table_idx,
                        }))
                        
                except Exception as e:
                    self.logger.warning(f"Error processing table {table_idx} on page {page_num}: {str(e)}")
                    continue
                    
        except Exception as e:
            self.logger.warning(f"Error extracting tables from page {page_num}: {str(e)}")
            
        return tables

In [1]:
import hashlib
import os
import json
import logging
import io
import pickle
from pathlib import Path
from typing import List, Dict, Any, Tuple, Optional, Union, Sequence
from datetime import datetime
from dataclasses import dataclass, asdict
import llama_index
import torch
import numpy as np
import cv2
from PIL import Image
import pymupdf as fitz
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from pydantic import BaseModel, Field
from llama_index.core.response_synthesizers.type import ResponseMode
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition

from pinecone import Pinecone, ServerlessSpec
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.llms.openai import OpenAI
from llama_index.readers.file import PDFReader
from llama_index.core import (
    VectorStoreIndex,
    Document,
    StorageContext,
    Settings,
    load_index_from_storage
)
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.schema import NodeWithScore
from llama_parse import LlamaParse

llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)
embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-base-en-v1.5",
            embed_batch_size=10
)
Settings.embed_model = embed_model
Settings.dimension = 768
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('document_processing.log'),
        logging.StreamHandler()
    ]
)


class TextBlock(BaseModel):
    """Text block in the report"""
    content_type: str = Field("text", description="Type of content block")  # Set default value
    text: str = Field(..., description="The text content")
    source: Optional[str] = Field(None, description="Source of the text")
    page_num: Optional[int] = Field(None, description="Page number")
    metadata: Optional[Dict] = Field(None, description="Additional metadata")


class ImageBlock():
    """Image block in the report"""
    content_type: str = Field("image", description="Type of content block")
    file_path: str = Field(..., description="Path to the image file")
    caption: Optional[str] = Field(None, description="Image caption")
    source: Optional[str] = Field(None, description="Source of the image")
    page_num: Optional[int] = Field(None, description="Page number")
    metadata: Optional[Dict] = Field(None, description="Additional metadata")


class TableBlock():
    """Table block in the report"""
    content_type: str = Field("table", description="Type of content block")
    file_path: str = Field(..., description="Path to the table image")
    excel_path: Optional[str] = Field(None, description="Path to Excel file")
    caption: Optional[str] = Field(None, description="Table caption")
    source: Optional[str] = Field(None, description="Source of the table")
    page_num: Optional[int] = Field(None, description="Page number")
    metadata: Optional[Dict] = Field(None, description="Additional metadata")


class ReportOutput():
    """Structured output for report generation"""
    title: str = Field(..., description="Report title")
    summary: str = Field(..., description="Executive summary")
    blocks: List[Union[TextBlock, ImageBlock, TableBlock]] = Field(
        ...,
        description="Content blocks in the report"
    )


def initialize_vector_store(index_name: str = "document-store") -> PineconeVectorStore:
    """Initialize Pinecone vector store with the new client."""

    # Initialize Pinecone client
    pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

    # Check if index exists, if not create it
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=768,  # OpenAI embedding dimension
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

    # Initialize vector store
    vector_store = PineconeVectorStore(
        index_name=index_name,
        environment=os.environ.get("PINECONE_ENVIRONMENT")
    )

    return vector_store


import logging
from pathlib import Path
from typing import List, Dict, Optional, Union
from dataclasses import dataclass
from pydantic import BaseModel, Field
import fitz
import torch
from transformers import AutoTokenizer, AutoModel
from IPython.display import display, Markdown, Image

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


class ContentBlock(BaseModel):
    """Base content block"""
    content_type: str
    page_num: int
    metadata: Dict


class TextBlock(ContentBlock):
    """Text block for output"""
    text: str = Field(..., description="The text content")


class ImageBlock(ContentBlock):
    """Image block for output"""
    image_path: str = Field(..., description="Path to image file")
    caption: Optional[str] = Field(None, description="Image caption")


class ReportOutput(BaseModel):
    """Structured report output"""
    title: str = Field(..., description="Report title")
    summary: str = Field(..., description="Executive summary")
    blocks: List[Union[TextBlock, ImageBlock]] = Field(
        ..., description="Content blocks"
    )

    def render(self):
        """Render report with text and images"""
        display(Markdown(f"# {self.title}"))
        display(Markdown(f"## Executive Summary\n{self.summary}"))

        for block in self.blocks:
            if isinstance(block, TextBlock):
                display(Markdown(block.text))
            else:
                display(Image(filename=block.image_path))
                if block.caption:
                    display(Markdown(f"*{block.caption}*"))


class DocumentProcessor:
    """Document processing with PyMuPDF"""

    def __init__(self, storage_dir: str = "./storage"):
        self.storage_dir = Path(storage_dir)
        self.storage_dir.mkdir(exist_ok=True)

        # Create subdirectories
        self.image_dir = self.storage_dir / "images"
        self.image_dir.mkdir(exist_ok=True)

        self.table_dir = self.storage_dir / "tables"
        self.table_dir.mkdir(exist_ok=True)

        self.cache_dir = self.storage_dir / "cache"
        self.cache_dir.mkdir(exist_ok=True)

        self.logger = logging.getLogger(__name__)

    def process_document(self, file_path: str) -> List[Document]:
        """Process document and return list of Documents"""
        cache_path = self.cache_dir / f"{Path(file_path).stem}_processed.pkl"

        if cache_path.exists():
            self.logger.info(f"Loading cached processing results: {cache_path}")
            import pickle
            with open(cache_path, 'rb') as f:
                return pickle.load(f)

        try:
            doc = fitz.open(file_path)
            all_documents = []

            for page_num in range(len(doc)):
                page = doc[page_num]
                page_documents = self._process_page(page, page_num, file_path)
                all_documents.extend(page_documents)

            doc.close()

            # Cache results
            with open(cache_path, 'wb') as f:
                import pickle
                pickle.dump(all_documents, f)

            return all_documents

        except Exception as e:
            self.logger.error(f"Error processing document: {str(e)}")
            raise

    def _process_page(self, page, page_num: int, source_file: str) -> List[Document]:
        """Process a single page and return list of Documents"""
        documents = []

        try:
            # Extract text blocks
            text_blocks = self._extract_text_blocks(page, page_num)
            for idx, block in enumerate(text_blocks):
                bboxstr = f"{block['bbox']['x0']},{block['bbox']['y0']},{block['bbox']['x1']},{block['bbox']['y1']}"
                doc = Document(
                    text=block["text"],
                    metadata={
                        "source": f"{Path(source_file).stem}-page{page_num}-text{idx}",
                        "page_num": page_num,
                        "type": "text",
                        "bbox": bboxstr,
                        "id_": f"{Path(source_file).stem}-page{page_num}-text{idx}"
                    }
                )
                documents.append(doc)

            # Extract images
            images = self._extract_images(page, page_num, source_file)
            for idx, image in enumerate(images):
                bbox = image["bbox"]
                bboxstr = f"{bbox['x1']},{bbox['y1']},{bbox['x2']},{bbox['y2']}"
                doc = Document(
                    text=f"Image from page {page_num}",
                    metadata={
                        "source": f"{Path(source_file).stem}-page{page_num}-image{idx}",
                        "page_num": page_num,
                        "type": "image",
                        "image_path": image["image_path"],
                        "bbox": bboxstr,
                        "id_": f"{Path(source_file).stem}-page{page_num}-image{idx}"
                    }
                )
                documents.append(doc)

            # Extract tables
            tables = self._extract_tables(page, page_num, source_file)
            for idx, table in enumerate(tables):
                bboxstr = f"{table['bbox'].x1},{table['bbox'].y1},{table['bbox'].x2},{table['bbox'].y2}"

                doc = Document(
                    text=f"Table from page {page_num}",
                    metadata={
                        "source": f"{Path(source_file).stem}-page{page_num}-table{idx}",
                        "page_num": page_num,
                        "type": "table",
                        "table_path": table["table_path"],
                        "excel_path": table["excel_path"],
                        "bbox": bboxstr,
                        "id_": f"{Path(source_file).stem}-page{page_num}-table{idx}"
                    }
                )
                documents.append(doc)

            return documents

        except Exception as e:
            self.logger.error(f"Error processing page {page_num}: {str(e)}")
            raise

    def _extract_text_blocks(self, page, page_num: int) -> List[Dict]:
        """Extract text blocks from page"""
        blocks = []
        for block in page.get_text("blocks"):
            if block[6] == 0:  # Text block
                blocks.append({
                    "text": block[4],
                    "bbox": {
                        "x0": block[0],
                        "y0": block[1],
                        "x1": block[2],
                        "y1": block[3]
                    },
                    "page_num": page_num
                })
        return blocks

    def _extract_images(self, page, page_num: int, source_file: str) -> List[Dict]:
        """Extract images from page"""
        images = []
        for img_index, img in enumerate(page.get_images(full=True)):
            try:
                xref = img[0]
                base_image = page.parent.extract_image(xref)

                if base_image:
                    image_path = self.image_dir / f"{Path(source_file).stem}_page{page_num}_img{img_index}.png"
                    with open(image_path, "wb") as f:
                        f.write(base_image["image"])
                    bboxstr = f"{img[1].x0},{img[1].y0},{img[1].x1},{img[1].y1}"
                    images.append(Document({
                        "image_path": str(image_path),
                        "bbox": bboxstr,
                        "page_num": page_num,
                        "id": img_index
                    }))
            except Exception as e:
                self.logger.warning(f"Error extracting image: {str(e)}")
                continue

        return images

    def _extract_tables(self, page, page_num: int, source_file: str) -> List[Dict]:
        """Extract tables from page"""
        tables = []
        try:
            found_tables = page.find_tables(
                horizontal_strategy="lines_strict",
                vertical_strategy="lines_strict"
            )

            for table_idx, table in enumerate(found_tables):
                try:
                    if not table.header.external:
                        # Convert to pandas DataFrame
                        pandas_df = table.to_pandas()

                        # Save table as Excel
                        excel_path = self.table_dir / f"{Path(source_file).stem}_page{page_num}_table{table_idx}.xlsx"
                        pandas_df.to_excel(str(excel_path))

                        # Create image of table area
                        table_bbox = fitz.Rect(table.bbox)
                        table_pix = page.get_pixmap(clip=table_bbox)
                        image_path = self.table_dir / f"{Path(source_file).stem}_page{page_num}_table{table_idx}.png"
                        table_pix.save(str(image_path))
                        bboxstr = f"{table.bbox.x0},{table.bbox.y0},{table.bbox.x1},{table.bbox.y1}"
                        tables.append(Document({
                            "table_path": str(image_path),
                            "excel_path": str(excel_path),
                            "bbox": bboxstr,
                            "columns": list(pandas_df.columns),
                            "id_": table_idx,
                        }))

                except Exception as e:
                    self.logger.warning(f"Error processing table {table_idx} on page {page_num}: {str(e)}")
                    continue

        except Exception as e:
            self.logger.warning(f"Error extracting tables from page {page_num}: {str(e)}")

        return tables

class RAGQueryEngine:
    """Enhanced RAG Query Engine with Pinecone vector store"""

    def __init__(
            self,
            storage_dir: str = "./storage",
            pinecone_api_key: str = os.getenv("PINECONE_API_KEY"),
            pinecone_environment: str = "gcp-starter",
            index_name: str = "rag-index"
    ):
        self.storage_dir = Path(storage_dir)
        self.cache_dir = self.storage_dir / "cache"
        self.logger = logging.getLogger(__name__)

        # Create directories
        self.storage_dir.mkdir(exist_ok=True)
        self.cache_dir.mkdir(exist_ok=True)

        # Initialize Pinecone
        self.pc = Pinecone(api_key=pinecone_api_key)
        self.index_name = index_name

        # Initialize embedding model
        self.embed_model = HuggingFaceEmbedding(
            model_name="BAAI/bge-base-en-v1.5",
            embed_batch_size=10
        )

        # Initialize LLM
        system_prompt = """You are a report generation assistant that produces well-formatted
        responses with text, images, and tables. Include relevant visual elements when they add value.
        Structure your response according to the ReportOutput schema."""

        self.llm = OpenAI(
            model="gpt-4o-mini",
            api_base="https://models.inference.ai.azure.com",
            system_prompt=system_prompt
        )

        # Set global models
        Settings.embed_model = self.embed_model
        Settings.llm = self.llm

        # Initialize or load Pinecone index
        self._initialize_vector_store()

    def _initialize_vector_store(self):
        """Initialize or load Pinecone vector store"""
        try:
            # Check if index exists
            existing_indexes = self.pc.list_indexes().names()

            if self.index_name not in existing_indexes:
                # Create new index
                self.pc.create_index(
                    name=self.index_name,
                    dimension=768,  # BGE-base dimension
                    metric="cosine",
                    spec=ServerlessSpec(
                        cloud="aws",
                        region="us-east-1"
                    )
                )
                self.logger.info(f"Created new Pinecone index: {self.index_name}")

            # Connect to index
            self.pinecone_index = self.pc.Index(self.index_name)

            # Initialize vector store
            self.vector_store = PineconeVectorStore(
                pinecone_index=self.pinecone_index,
                metadata_filters={"type": ["text", "image", "table"]}
            )


            # Initialize storage context
            self.storage_context = StorageContext.from_defaults(
                vector_store=self.vector_store
            )

        except Exception as e:
            self.logger.error(f"Error initializing vector store: {str(e)}")
            raise

    def _get_cache_key(self, documents: List[Document]) -> str:
        """Generate cache key for documents"""
        content_hash = hashlib.md5()
        for doc in sorted(documents, key=lambda x: x.id_):
            content_hash.update(str(doc.hash).encode())
        return content_hash.hexdigest()

    def build_index(self, documents: List[Document]):
        """Build or load index with caching"""
        try:
            # Generate cache key
            cache_key = self._get_cache_key(documents)
            cache_path = self.cache_dir / f"index_cache_{cache_key}.pkl"

            # Check cache
            if cache_path.exists():
                self.logger.info("Loading index from cache...")
                with open(cache_path, 'rb') as f:
                    cached_data = pickle.load(f)

                # Verify if documents are already in Pinecone
                stats = self.pinecone_index.describe_index_stats()
                if stats.total_vector_count == len(documents):
                    self.logger.info("Documents already indexed in Pinecone")
                    return

            # Build new index
            self.logger.info("Building new index...")

            # Prepare documents for upsert
            vectors_to_upsert = []
            for doc in documents:
                # Generate embedding
                embedding = self.embed_model.get_text_embedding(doc.text)

                # Prepare vector data
                vector_data = {
                    "id": doc.id_,
                    "values": embedding,
                    "metadata": {
                        **doc.metadata,
                        "text": doc.text,
                        "doc_hash": doc.hash
                    }
                }
                vectors_to_upsert.append(vector_data)

            # Upsert in batches
            batch_size = 100
            for i in range(0, len(vectors_to_upsert), batch_size):
                batch = vectors_to_upsert[i:i + batch_size]
                self.pinecone_index.upsert(vectors=batch)

            # Cache the index metadata
            with open(cache_path, 'wb') as f:
                pickle.dump({
                    'doc_hashes': [doc.hash for doc in documents],
                    'index_name': self.index_name
                }, f)

            self.logger.info(f"Successfully indexed {len(documents)} documents")

        except Exception as e:
            self.logger.error(f"Error building index: {str(e)}")
            raise

    async def query(
            self,
            query_text: str,
            response_mode: str = "tree_summarize",
            similarity_top_k: int = 5
    ) -> ReportOutput:
        """Execute structured query"""
        try:
            # Create structured LLM
            structured_llm = self.llm.as_structured_llm(output_cls=ReportOutput)

            # Configure response synthesizer
            response_synthesizer = get_response_synthesizer(
                response_mode=ResponseMode.TREE_SUMMARIZE,
                structured_answer_filtering=True
            )
            index = VectorStoreIndex.from_vector_store(vector_store=self.vector_store, show_progress=True)
            # Create query engine
            query_engine = index.as_query_engine(
                similarity_top_k=similarity_top_k,
                response_synthesizer=response_synthesizer,
                llm=structured_llm,
                node_postprocessors=[
                    SimilarityPostprocessor(similarity_cutoff=0.7)
                ]
            )

            # Execute query
            response = await query_engine.aquery(query_text)

            # Process and structure the response
            return self._process_response(response)

        except Exception as e:
            self.logger.error(f"Error executing query: {str(e)}")
            raise

    def _process_response(self, response) -> ReportOutput:
        """Process query response into structured output"""
        blocks = []

        # Process source nodes
        for node in response.source_nodes:
            metadata = node.metadata

            if metadata.get("type") == "text":
                blocks.append(TextBlock(
                    text=node.text,
                    source=metadata.get("source"),
                    page_num=metadata.get("page_num"),
                    metadata=metadata,
                    content_type="text"
                ))

            elif metadata.get("type") == "image":
                blocks.append(ImageBlock(
                    file_path=metadata.get("image"),
                    caption=metadata.get("caption"),
                    source=metadata.get("source"),
                    page_num=metadata.get("page_num"),
                    metadata=metadata,
                    content_type="image"

                ))

            elif metadata.get("type") == "table":
                blocks.append(TableBlock(
                    file_path=metadata.get("image"),
                    excel_path=metadata.get("dataframe"),
                    caption=metadata.get("caption"),
                    source=metadata.get("source"),
                    page_num=metadata.get("page_num"),
                    content_type="table",
                    metadata=metadata
                ))
        mcd = response
        # if empty response throw error

        logging.info(f"Response: {response}")
        return ReportOutput(
            title=f"Report",
            summary=response.response,
            blocks=blocks
        )

    def render_report(self, report: ReportOutput):
        """Render the structured report"""
        from IPython.display import display, Markdown, Image

        # Display title and summary
        display(Markdown(f"# {report.title}"))
        display(Markdown(f"## Executive Summary\n{report.summary}\n"))

        # Display content blocks
        for block in report.blocks:
            if isinstance(block, TextBlock):
                display(Markdown(f"### Content from {block.metadata["source"]} (Page {block.page_num})\n{block.text}\n"))

            elif isinstance(block, ImageBlock):
                display(Markdown(f"### Image from {block.metadata["source"]} (Page {block.page_num})"))
                display(Image(filename=block.file_path))
                if block.caption:
                    display(Markdown(f"*{block.caption}*\n"))

            elif isinstance(block, TableBlock):
                display(Markdown(f"### Table from {block.metadata["source"]} (Page {block.page_num})"))
                display(Image(filename=block.file_path))
                if block.caption:
                    display(Markdown(f"*{block.caption}*\n"))
                display(Markdown(f"[Download Excel]({block.excel_path})\n"))


async def main():
    # Initialize query engine
    query_engine = RAGQueryEngine()
    processor = DocumentProcessor()
    # Load and process documents
    documents = processor.process_document(
        "/Users/saisuryamadhav/Documents/University/new/Assignment_3_Team1/backend/pdf/sample.pdf")

    # Build index
    query_engine.build_index(documents)

    # Execute query
    # query = "Summarize the key financial metrics and include relevant charts"
    query = "What is this assignment about?"
    report = await query_engine.query(
        query,
        response_mode="tree_summarize",
        similarity_top_k=10
    )

    # Render report
    query_engine.render_report(report)

import nest_asyncio
import asyncio

# Apply nest_asyncio to allow nested event loops (needed for Jupyter)
nest_asyncio.apply()
if __name__ == "__main__":
    asyncio.run(main())

2024-11-04 15:57:39,764 - pinecone_plugin_interface.logging - INFO - Discovering subpackages in _NamespacePath(['/Users/saisuryamadhav/Library/Caches/pypoetry/virtualenvs/backend-dSPC7ywe-py3.12/lib/python3.12/site-packages/pinecone_plugins'])
2024-11-04 15:57:39,767 - pinecone_plugin_interface.logging - INFO - Looking for plugins in pinecone_plugins.inference
2024-11-04 15:57:39,798 - pinecone_plugin_interface.logging - INFO - Installing plugin inference into Pinecone
2024-11-04 15:57:39,800 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
2024-11-04 15:57:40,817 - sentence_transformers.SentenceTransformer - INFO - 2 prompts are loaded, with the keys: ['query', 'text']
2024-11-04 15:57:42,466 - __main__ - INFO - Building new index...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-11-04 15:58:01,899 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"
2024-11-04 15:58:02,262 - root - INFO - Response: # Assignment Overview Report

## Assignment Title: Assignment 3

### Description
The assignment appears to be a structured task that requires the integration of information from multiple sources. The specific details regarding the content, objectives, or requirements of the assignment are not provided in the available information.

### Key Points
- **Integration of Sources**: The assignment likely involves synthesizing information from various texts or documents.
- **Focus on Analysis**: There may be an emphasis on critical thinking and analysis of the provided materials.

### Conclusion
Further details regarding the specific objectives, guidelines, or topics covered in Assignment 3 would be necessary to provide a comprehensive understanding of its requirements.


# Report

## Executive Summary
# Assignment Overview Report

## Assignment Title: Assignment 3

### Description
The assignment appears to be a structured task that requires the integration of information from multiple sources. The specific details regarding the content, objectives, or requirements of the assignment are not provided in the available information.

### Key Points
- **Integration of Sources**: The assignment likely involves synthesizing information from various texts or documents.
- **Focus on Analysis**: There may be an emphasis on critical thinking and analysis of the provided materials.

### Conclusion
Further details regarding the specific objectives, guidelines, or topics covered in Assignment 3 would be necessary to provide a comprehensive understanding of its requirements.


### Content from sample-page0-text0 (Page 0)
Assignment 3



In [9]:
    processor = DocumentProcessor()
    # Load and process documents
    documents = processor.process_document("/Users/saisuryamadhav/Documents/University/new/Assignment_3_Team1/backend/pdf/Beyond Active and Passive Investing_ The Customization of Finance.pdf")
    

2024-11-04 02:05:13,314 - __main__ - INFO - Loading cached processing results: storage/cache/Beyond Active and Passive Investing_ The Customization of Finance_processed.pkl


In [41]:
print(mcd)

None


In [46]:

llm = OpenAI(model="gpt-4o-mini", api_base="https://models.inference.ai.azure.com")

In [47]:
llm.complete("Summarize the key financial metrics and include relevant charts")

2024-11-04 10:14:22,333 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


CompletionResponse(text="I currently don't have the capability to create or display charts directly. However, I can help you summarize key financial metrics and guide you on how to create charts based on that data. \n\nTo summarize key financial metrics, you typically want to focus on the following areas:\n\n1. **Revenue**: Total income generated from sales or services.\n2. **Net Income**: Profit after all expenses, taxes, and costs have been deducted from revenue.\n3. **Gross Margin**: (Revenue - Cost of Goods Sold) / Revenue, expressed as a percentage.\n4. **Operating Margin**: Operating Income / Revenue, expressed as a percentage.\n5. **Net Profit Margin**: Net Income / Revenue, expressed as a percentage.\n6. **Earnings Per Share (EPS)**: Net Income / Number of Outstanding Shares.\n7. **Return on Equity (ROE)**: Net Income / Shareholder's Equity, expressed as a percentage.\n8. **Debt-to-Equity Ratio**: Total Liabilities / Shareholder's Equity, indicating financial leverage.\n9. **Cu